In [1]:
import populartimes
import os
import sys
from dotenv import load_dotenv
from pathlib import Path
env_path = str(Path('.') / '.env')
load_dotenv(dotenv_path=env_path)

import pandas as pd
import numpy as np
import math
from time import sleep

### Load station data & add data fields

In [2]:
station_output = str(Path('.') / './dataframes/station_output_2017.csv')
# headers = ['id', url, name	plug_score	ports	address	phone	is_free	cost_desc	hours	description	latitude	longitude, 'type', 'created_at', 'local'e]
stationDF = pd.read_csv(station_output,  sep='	', na_values=['NaN'])

In [3]:
stationDF.head()

,id,url,name,plug_score,ports,address,phone,is_free,cost_desc,hours,description,latitude,longitude,type,created_at,locale
0,3,http://api.plugshare.com/view/location/3,Adams/Wabash - Self-Park Garage,5.9,"ChargePoint: Wall Outlet (120V), EV Plug (J177...","17 E Adams St, Chicago IL 60603",(888) 758-4389,No,First 3 hours free. $2/hr after that.,24/7,On right-hand side as you turn on Level 2.,41.879495,-87.626966,G,2011-08-17T00:05:00Z,US
1,5,http://api.plugshare.com/view/location/5,Aspira Seattle Apartment Homes,7.0,"ChargePoint: Wall Outlet (120V), EV Plug (J177...","1823 Terry Ave., Seattle, WA, 98101",(206) 623-8900,Yes,NaN,NaN,Aspira Resident Parking Only,47.615963,-122.333477,B,2011-08-17T00:05:00Z,US
2,6,http://api.plugshare.com/view/location/6,Marc Geller,9.0,"NEMA 14-50, Wall Outlet (120V), EV Plug (J1772)",847 Haight St San Francisco,(415) 336-5678,Yes,NaN,NaN,NaN,37.771370,-122.436203,H,2011-02-28T07:49:39Z,US
3,9,http://api.plugshare.com/view/location/9,Gary Savage,NaN,"Wall Outlet (120V), EV Plug (J1772)","530 Mt Pleasant Ave, Dover, NJ 07801, USA",(973) 349-8805,Yes,NaN,NaN,NaN,40.903189,-74.563347,H,2011-03-21T11:33:39Z,US
4,10,http://api.plugshare.com/view/location/10,Mitch Albert,NaN,EV Plug (J1772),"8016 E Roper St, Long Beach, CA 90808, USA",(562) 572-3740,Yes,NaN,NaN,Please text first 120v,33.816231,-118.074226,H,2011-02-03T05:56:47Z,US


#### create new columns

In [4]:
def filterPorts(x):
    port_count = len(str(x).replace('|', ',').split(','))
    return port_count

def coord_p2(x):
    p2 = float(x) + 0.005
    return p2

def coord_p1(x):
    p1 = float(x) - 0.005
    return p1

In [5]:
stationDF['port_count'] = stationDF['ports'].apply(filterPorts)
stationDF['p1_latitude'] = stationDF['latitude'].apply(coord_p1)
stationDF['p1_longitude'] = stationDF['longitude'].apply(coord_p1)
stationDF['p2_latitude'] = stationDF['latitude'].apply(coord_p2)
stationDF['p2_longitude'] = stationDF['longitude'].apply(coord_p2)

In [6]:
stationDF.head()

,id,url,name,plug_score,ports,address,phone,is_free,cost_desc,hours,...,latitude,longitude,type,created_at,locale,port_count,p1_latitude,p1_longitude,p2_latitude,p2_longitude
0,3,http://api.plugshare.com/view/location/3,Adams/Wabash - Self-Park Garage,5.9,"ChargePoint: Wall Outlet (120V), EV Plug (J177...","17 E Adams St, Chicago IL 60603",(888) 758-4389,No,First 3 hours free. $2/hr after that.,24/7,...,41.879495,-87.626966,G,2011-08-17T00:05:00Z,US,4,41.874495,-87.631966,41.884495,-87.621966
1,5,http://api.plugshare.com/view/location/5,Aspira Seattle Apartment Homes,7.0,"ChargePoint: Wall Outlet (120V), EV Plug (J177...","1823 Terry Ave., Seattle, WA, 98101",(206) 623-8900,Yes,NaN,NaN,...,47.615963,-122.333477,B,2011-08-17T00:05:00Z,US,4,47.610963,-122.338477,47.620963,-122.328477
2,6,http://api.plugshare.com/view/location/6,Marc Geller,9.0,"NEMA 14-50, Wall Outlet (120V), EV Plug (J1772)",847 Haight St San Francisco,(415) 336-5678,Yes,NaN,NaN,...,37.771370,-122.436203,H,2011-02-28T07:49:39Z,US,3,37.766370,-122.441203,37.776370,-122.431203
3,9,http://api.plugshare.com/view/location/9,Gary Savage,NaN,"Wall Outlet (120V), EV Plug (J1772)","530 Mt Pleasant Ave, Dover, NJ 07801, USA",(973) 349-8805,Yes,NaN,NaN,...,40.903189,-74.563347,H,2011-03-21T11:33:39Z,US,2,40.898189,-74.568347,40.908189,-74.558347
4,10,http://api.plugshare.com/view/location/10,Mitch Albert,NaN,EV Plug (J1772),"8016 E Roper St, Long Beach, CA 90808, USA",(562) 572-3740,Yes,NaN,NaN,...,33.816231,-118.074226,H,2011-02-03T05:56:47Z,US,1,33.811231,-118.079226,33.821231,-118.069226


### 1. Get busy times for each station
#### create dataframes for each day of aggregrate busy-ness within a .1 degree radius

In [7]:
# duplicate dataframe
# busy_column = stationDF

In [8]:
# Limit to Massachussets
ub_lat = 42.7248
lb_lat = 42.03705
ub_long = -70.5432
lb_long = -72.1403

In [9]:
# stationDF.loc[stationDF['column_name'] == some_value]
# df.loc[(df['column_name'] == some_value) & df['other_column'].isin(some_values)]
maDF = stationDF.loc[(stationDF['latitude'] < ub_lat) & (stationDF['latitude'] > lb_lat) & (stationDF['longitude'] < ub_long) & (stationDF['longitude'] > lb_long)]

In [10]:
group = [pd.DataFrame(columns=pd.Series(range(0,24)))]*7;

In [11]:
api_key = os.getenv("GOOGLE_API_KEY")
types = ["establishment"]

In [12]:
def calculate_composite(row, index):
    if index > 93879:
        station_id = row['id']
        station_index = index
        composite_array = [[0] * 24] * 7
        p1 = (row['p1_latitude'], row['p1_longitude'])
        p2 = (row['p2_latitude'], row['p2_longitude'])

        alt_id = populartimes.get(api_key, types, p1, p2, n_threads=12)

        # Calculate composite array for busy times
        for location in alt_id:
            for x in range(7):
                try:
                    composite_array[x] = [composite_array[x][i] + location['populartimes'][x]['data'][i] for i in range(24)]
                except:
                    print('error processing item')
                    pass

        # Create dataframes for each iteam
        for i in range(7):
            day_series = pd.DataFrame(composite_array[i:i+1])
            day_series['id'] = station_id
            day_series['index'] = index
            group[i] = group[i].append(day_series)

        print('processed index item: ', group[1])
        return


In [16]:
for i in range(7):
    with open(str(Path('.') / './dataframes/weekly/061818_{}.csv'.format(i)), 'a') as f:
        group[i].to_csv(f, mode='a', header=False)

In [14]:
# group[0]

In [15]:
for index, row in maDF.iterrows(): # change to stationDF for full list
    calculate_composite(row, index)
    print ('processed index / row: ', index)

processed index / row:  16
processed index / row:  37
processed index / row:  132
processed index / row:  162
processed index / row:  201
processed index / row:  214
processed index / row:  235
processed index / row:  315
processed index / row:  336
processed index / row:  426
processed index / row:  446
processed index / row:  501
processed index / row:  538
processed index / row:  544
processed index / row:  555
processed index / row:  558
processed index / row:  622
processed index / row:  1123
processed index / row:  1126
processed index / row:  1127
processed index / row:  1128
processed index / row:  1131
processed index / row:  1133
processed index / row:  1453
processed index / row:  1559
processed index / row:  1880
processed index / row:  1917
processed index / row:  1920
processed index / row:  2197
processed index / row:  2543
processed index / row:  2643
processed index / row:  2672
processed index / row:  2689
processed index / row:  2747
processed index / row:  2940
proc

//anaconda/lib/python3.5/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: unorderable types: int() < str(), sort order is undefined for incomparable objects
  result = result.union(other)


processed index item:      0  1  2  3   4   5   6    7    8     9   ...       16   17   18   19   20  \
0  11  3  3  3  23  43  99  250  570  1033   ...     1035  902  831  644  410   

    21  22  23        id    index  
0  165  26  17  126952.0  94246.0  

[1 rows x 26 columns]
processed index / row:  94246
processed index item:       0    1   2   3   4    5    6    7    8     9   ...       16    17    18  \
0   11    3   3   3  23   43   99  250  570  1033   ...     1035   902   831   
0  312  190  83  44  18  114  239  391  939  1366   ...     2547  2937  3312   

     19    20    21    22   23        id    index  
0   644   410   165    26   17  126952.0  94246.0  
0  3159  2478  1868  1269  723  127020.0  94306.0  

[2 rows x 26 columns]
processed index / row:  94306
processed index item:       0    1   2   3   4    5    6    7    8     9   ...       16    17    18  \
0   11    3   3   3  23   43   99  250  570  1033   ...     1035   902   831   
0  312  190  83  44  18  114  239

processed index item:       0    1   2   3   4    5    6     7     8     9   ...       16    17  \
0   11    3   3   3  23   43   99   250   570  1033   ...     1035   902   
0  312  190  83  44  18  114  239   391   939  1366   ...     2547  2937   
0  106   21   0   0   0    7   76   226   511   878   ...     1618  1770   
0    0    0   0   0   0    0    0     0    90    63   ...       81     0   
0   27    8   8   8   2    5  113   297   490   778   ...     1113  1128   
0    0    0   0   0   0    0    0     0     0     0   ...        0     0   
0    0    0   0   0   0    0    0     0    69    80   ...       46    28   
0  357  163  42  43  33  115  464  1194  1974  2785   ...     6572  6790   
0    0    0   0   0   0    0    0     0     0     0   ...       12    29   
0  385  212  74  48  32  100  398   947  2666  4614   ...     6647  6461   

     18    19    20    21    22   23        id    index  
0   831   644   410   165    26   17  126952.0  94246.0  
0  3312  3159  2478  186

In [38]:
len([0,0])

2

### 2. Create number of ports dataframe
#### same size as the busy-ness dataframes to be able to do matrix multiplication

In [26]:
ports_df = pd.DataFrame(columns=pd.Series(range(0,24)))

for i in range(24):
    ports_df[i] = stationDF['port_count']

with open(str(Path('.') / './dataframes/ports_df.csv'), 'a') as f:
    ports_df.to_csv(f, mode='a', header=False)


In [19]:
# stationDF['address']

In [13]:
# stationDF.loc[stationDF['column_name'] == some_value]

### 3. Create quality of port dataframe
#### turn this into a percentage scalar to adjust demand based on perceived quality (div by 10)

In [38]:
# create percentage quality score
stationDF['quality_score'] = stationDF['plug_score'] / 10
stationDF['quality_score'] = stationDF['quality_score'].fillna(value=1)
# print(stationDF['quality_score'])

In [36]:
quality_df = pd.DataFrame(columns=pd.Series(range(0,24)))

for i in range(24):
    quality_df[i] = stationDF['quality_score']

with open(str(Path('.') / './dataframes/quality_df.csv'), 'a') as f:
    quality_df.to_csv(f, mode='a', header=False)

In [40]:
# save stationDF as new CSV to reduce overhead
with open(str(Path('.') / './dataframes/station_updated_db.csv'), 'a') as f:
    stationDF.to_csv(f, mode='a', header=True)

In [32]:
with open(str(Path('.') / './dataframes/ma_df.csv'), 'a') as f:
    maDF.to_csv(f, mode='a', header=True)

### 4. Create file for US EV sales by car / plug type
#### use this for determining specific port busy-ness granularity

In [26]:
ev_sales = str(Path('.') / './dataframes/2017_ev_sales_by_mfg.csv')
ev_sales_df = pd.read_csv(ev_sales,  sep=',', na_values=['NaN'])
ev_sales_df

,BRAND,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,TOTAL,MARKET_SHARE
0,Tesla Model S*,900,1750,3450,1125,1620,2350,1425,2150,4860,1120,1335,4975,"27,060",0.1354
1,Chevrolet Bolt EV,1162,952,978,1292,1566,1642,1971,2107,2632,2781,2987,3227,"23,297",0.1166
2,Tesla Model X*,750,800,2750,715,1730,2200,1650,1575,3120,850,1875,3300,"21,315",0.1067
3,Toyota Prius Prime,1366,1362,1618,1819,1908,1619,1645,1820,1899,1626,1834,2420,"20,936",0.1048
4,Chevrolet Volt,1611,1820,2132,1807,1817,1745,1518,1445,1453,1362,1702,1937,"20,349",0.1018
5,Nissan LEAF,772,1037,1478,1063,1392,1506,1283,1154,1055,213,175,102,"11,230",0.0562
6,Ford Fusion Energi,606,837,1002,905,1000,707,703,762,763,741,731,875,"9,632",0.0482
7,Ford C-Max Energi,473,639,662,720,950,936,844,705,683,569,523,436,"8,140",0.0407
8,BMW i3 (BEV + REx),382,318,703,516,506,567,601,504,538,686,283,672,"6,276",0.0314
9,Fiat 500e**,752,590,785,541,473,359,395,290,285,310,215,385,"5,380",0.0269


### 5. Calculate composite dataframe 
#### multiply each data frame 1-3 above to yield dataframe 4

### 6. Save dataframes to AWS
#### test API calls to get availability at current time or a specific time

### Sandbox

In [18]:
# Get Google data for an address and compare versus Lat / Lng 

In [19]:
# See what is returned for that address

# See what types are returned


In [200]:
print(len(alt_id))

1


In [ ]:
# alt_id[ESTABLISHMENT]['populartimes'][DAY_OF_WEEK]['data]
# returns an array of 24 times
# print(alt_id[0]['populartimes'][0]['data'])

In [ ]:
# print(composite_array[6])
# a = [[0] * 24] * 7
# print(alt_id[0]['populartimes'][0]['data'])
# print(alt_id[0]['populartimes'][0]['data'][24])
# print(alt_id[0]['populartimes'][0]['data'][25])

In [ ]:
# google_places = GooglePlaces(api_key)

# query_result = google_places.nearby_search(location=test_address)
# for place in query_result.places:
#     print (place.name)
#     print (place.geo_location)
#     print (place.place_id)

# for place in query_result.places:
#     place.get_details()
#     print (place.rating)

In [7]:
# api_key = os.getenv("GOOGLE_API_KEY")
# test_address = "1018 Campus Drive, Stanford, CA 94305, USA"
# test_address_2 = "125 Precita Ave, Moss Beach, CA 94038, USA"
# test_address_3 = "425 S Main St, Ann Arbor MI 48104"
# types = ["establishment"]
# p1 = (37.220627, -122.369142)
# p2 = (37.230627, -122.359142)

In [8]:
# test_name = "Boulder Nissan"
# p1_name = (40.015141, -105.278906)
# p2_name = (40.025141, -105.268906)

# min(p1_name[0], p2_name[0])

In [9]:
# alt_id = populartimes.get(api_key, types, p1_name, p2_name)

In [10]:
# composite_array = [[0] * 24] * 7

In [11]:
# for location in alt_id:
#     for x in range(7):
#     #     composite_array = [a[i]+b[i] for i in range(len(a))]
#         try:
#             composite_array[x] = [composite_array[x][i] + location['populartimes'][x]['data'][i] for i in range(24)]
#         except:
#             print('error processing item')
#             pass

In [12]:
# print(composite_array)

In [13]:
# print(alt_id[7]['populartimes'][6]['data'])

# df_test = pd.DataFrame()

In [14]:
# df_test.append([[1,0,0,0], [2,0,0,0]])

In [14]:
# api_key = os.getenv("GOOGLE_API_KEY")
# types = ["establishment"]
# p1_name = [40.015141, -105.278906]
# p2_name = [40.025141, -105.268906]
# composite_array = [[0] * 24] * 7

# alt_id = populartimes.get(api_key, types, p1_name, p2_name)

In [15]:
# for location in alt_id:
#     for x in range(7):
#     #     composite_array = [a[i]+b[i] for i in range(len(a))]
#         try:
#             composite_array[x] = [composite_array[x][i] + location['populartimes'][x]['data'][i] for i in range(24)]
#             print(composite_array[x])
#         except:
#             print('error processing item')
#             pass

In [16]:
# for i in range(7):
#     day_series = pd.DataFrame(composite_array[i:i+1])
#     day_series['id'] = 3
#     group[i] = group[i].append(day_series)

In [17]:
# print('processed item, current group[1] is: ', group[1])